In [1]:
import pandas as pd
from transformers import BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import torch
from torch import nn
from sklearn.metrics import accuracy_score, auc
from sklearn.model_selection import train_test_split
from transformers import BertConfig, BertForPreTraining
import os
import random
def seed_everything(seed = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.set_deterministic(True)
    torch.cuda.manual_seed_all(seed)
#     torch.backends.cudnn.deterministic = True
seed_everything()
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
train = pd.read_csv('gaiic_track3_round1_train_20210228.tsv',sep='\t', names=['text_a', 'text_b', 'label'])
test = pd.read_csv('gaiic_track3_round1_testA_20210228.tsv',sep='\t', names=['text_a', 'text_b', 'label'])
test['label'] = 0

In [3]:
##查看训练集和测试集中字
from collections import defaultdict
def get_dict(data):
    words_dict = defaultdict(int)
    for i in tqdm(range(data.shape[0])):
        text = data.text_a.iloc[i].split() + data.text_b.iloc[i].split()
        for c in text:
            words_dict[c] += 1
    return words_dict
test_dict = get_dict(test)
train_dict = get_dict(train)
word_dict = list(test_dict.keys()) + list(train_dict.keys())
word_dict = set(word_dict)
word_dict = set(map(int, word_dict))
word_dict = list(word_dict)
special_tokens = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
WORDS = special_tokens + word_dict
pd.Series(WORDS).to_csv('Bert-vocab.txt', header=False,index=0)
vocab = pd.read_csv('Bert-vocab.txt', names=['word'])
vocab_dict = {}
for key, value in vocab.word.to_dict().items():
    vocab_dict[value] = key

100%|██████████| 100000/100000 [00:02<00:00, 35187.95it/s]


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = BertConfig(vocab_size=len(WORDS)+1)
model = BertForPreTraining(config)
model.load_state_dict(torch.load('test.pth',  map_location=device))

<All keys matched successfully>

In [5]:
class BERTModel(nn.Module):
    def __init__(self, model:BertForPreTraining):
        super(BERTModel, self).__init__()
        self.model = model.bert
        self.transform = model.cls.predictions.transform
        self.linear = nn.Linear(in_features=768, out_features=2)
    def forward(self, inputs_ids, token_type_ids, attention_mask):
        x = self.model(inputs_ids, token_type_ids, attention_mask)
        pooler_output = x.pooler_output
        last_hidden_state = x.last_hidden_state
#         x = self.transform(x)
        
        x = self.linear(pooler_output)
        return x

In [6]:
class OPPODataset(Dataset):
    def __init__(self, data, word_dict,seq_length=50):
        '''
        data:dataFrame()
        '''
        self.data = data
        self.vocab = word_dict
        self.seq_len = seq_length
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, index):
        '''
        transformers 中可以用BertTokenizer实现下面的方法但是有一些不灵活
        transoformers格式
        0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence |
        '''
        text_a, text_b, label = self.data.iloc[index].values
        text_a = self.get_sentence(text_a)
        text_b = self.get_sentence(text_b)
        text_a = [self.vocab['[CLS]']] + text_a + [self.vocab['[SEP]']]
        text_b = text_b + [self.vocab['[SEP]']]

        token_type_ids = ([0 for _ in range(len(text_a))] + [1 for _ in range(len(text_b))])[:self.seq_len]
        text = (text_a + text_b)[:self.seq_len]

        padding = [self.vocab['[PAD]'] for _ in range(self.seq_len - len(text))]
        attention_mask = len(text) * [1]
        
        text.extend(padding), token_type_ids.extend(padding), attention_mask.extend(padding)
        attention_mask = np.array(attention_mask)
        text = np.array(text)
        token_type_ids = np.array(token_type_ids)
#         print(text.shape, token_type_ids.shape, attention_mask.shape)
        return {
                'input_ids': text,
                'token_type_ids': token_type_ids,
                'attention_mask': attention_mask,
               }, self.data.label.iloc[index]
    def get_sentence(self, sentence):
        tokens = sentence.split()
        for i in range(len(tokens)):
            tokens[i] = self.vocab.get(tokens[i], self.vocab['[UNK]'])
        return tokens

In [7]:
train_index, valid_index = train_test_split(range(train.shape[0]), test_size=0.2)
train_dataset = OPPODataset(train.iloc[train_index],vocab_dict,64)
valid_dataset = OPPODataset(train.iloc[valid_index],vocab_dict, 64)
test_dataset = OPPODataset(test, vocab_dict, 64)

In [8]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [9]:
def evaluate(model, data_loader,  device='cuda'):
    model.eval()
    losses = []
    labels_list = []
    preds_list = []
    for data_labels in tqdm(data_loader):
        data = data_labels[0]
        labels = data_labels[1]
        inputs_ids = data['input_ids'].to(device).long()
        token_type_ids = data['token_type_ids'].to(device).long()
        attention_mask = data['attention_mask'].to(device).long()
        preds = model(inputs_ids, token_type_ids, attention_mask)
        preds = torch.softmax(preds, dim=-1)
        preds_list.append(preds.argmax(-1).cpu().detach().numpy())
        labels_list.append(labels.cpu().detach().numpy())
#         break
#         print(preds, labels)
    results = np.concatenate(preds_list)
    labels = np.concatenate(labels_list)
    acc = accuracy_score(labels, results)
    return acc, results, labels

In [12]:
nums_epoch = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'
BertModel = BERTModel(model)
BertModel = BertModel.to(device)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
optim = torch.optim.Adam(BertModel.parameters(), lr=2e-5)

In [ ]:
best_acc = 0
for epoch in range(nums_epoch):
    BertModel.train()
    losses = []
    pbar = tqdm(train_loader)
    for data_labels in pbar:
        data = data_labels[0]
        labels = data_labels[1]
        inputs_ids = data['input_ids'].to(device).long()
        token_type_ids = data['token_type_ids'].to(device).long()
        attention_mask = data['attention_mask'].to(device).long()
        optim.zero_grad()
        labels =labels.to(device).long()
        preds = BertModel(inputs_ids, token_type_ids, attention_mask)
        loss = criterion(preds, labels)
        losses.append(loss.cpu().detach().numpy())
        loss.backward()
        optim.step()
        pbar.set_description(f'epoch:{epoch} loss:{np.mean(losses)}')
    valid_acc=evaluate(BertModel, valid_loader)
    print('=*'*50)
    print(f'epoch:{epoch}, valid_acc{valid_acc}')
    print('=*'*50)
    if valid_acc[0] > best_acc:
        best_acc = valid_acc[0]
        torch.save(BertModel.state_dict(), f'BertModel.pth', _use_new_zipfile_serialization=False)

100%|██████████| 313/313 [00:35<00:00,  8.81it/s]


=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
epoch:0, valid_acc(0.73905, array([0, 0, 1, ..., 0, 0, 0]), array([0, 0, 0, ..., 1, 1, 0]))
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*


100%|██████████| 313/313 [00:35<00:00,  8.83it/s]


=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*
epoch:1, valid_acc(0.74255, array([0, 0, 1, ..., 0, 1, 0]), array([0, 0, 0, ..., 1, 1, 0]))
=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*


epoch:2 loss:0.45609021186828613:   2%|▏         | 20/1250 [00:06<06:57,  2.95it/s]